# Azure audio whisper (プレビュー) の例

この例では、Azure OpenAI Whisperモデルを使用してオーディオファイルを文字起こしする方法を示します。

## セットアップ

まず、必要な依存関係をインストールし、使用するライブラリをインポートします。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 認証

Azure OpenAIサービスは、APIキーとAzure Active Directoryトークン認証情報を含む複数の認証メカニズムをサポートしています。

In [4]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下にある*「キーとエンドポイント」*で確認できます）。リソースのエンドポイントもここで確認できます。

In [5]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Azure Active Directoryを使用した認証
次に、Azure Active Directoryを使用して認証する方法を見てみましょう。まず、`azure-identity`ライブラリをインストールします。このライブラリは、認証に必要なトークン認証情報を提供し、`get_bearer_token_provider`ヘルパー関数を通じてトークン認証情報プロバイダーの構築を支援します。静的トークンを`AzureOpenAI`に提供するよりも`get_bearer_token_provider`を使用することが推奨されます。なぜなら、このAPIは自動的にトークンをキャッシュし、更新してくれるからです。

Azure OpenAIでAzure Active Directory認証を設定する方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意: AzureOpenAIは、以下の引数が提供されていない場合、対応する環境変数から推測します：

- `api_key` は `AZURE_OPENAI_API_KEY` から
- `azure_ad_token` は `AZURE_OPENAI_AD_TOKEN` から
- `api_version` は `OPENAI_API_VERSION` から
- `azure_endpoint` は `AZURE_OPENAI_ENDPOINT` から

## デプロイメント

このセクションでは、`whisper-1`モデルを使用してオーディオファイルを転写するデプロイメントを作成します。

### デプロイメント: Azure OpenAI Studioでの作成
whisperで使用するモデルをデプロイしましょう。https://portal.azure.com にアクセスし、Azure OpenAIリソースを見つけて、Azure OpenAI Studioに移動します。「Deployments」タブをクリックし、whisperで使用したいモデルのデプロイメントを作成します。モデルに付けるデプロイメント名は、以下のコードで使用されます。

In [6]:
deployment = "whisper-deployment" # Fill in the deployment name from the portal here

## 音声転写

音声転写、または音声テキスト変換は、話された言葉をテキストに変換するプロセスです。音声ファイルストリームをテキストに転写するには、`openai.Audio.transcribe`メソッドを使用してください。

サンプル音声ファイルは、[GitHubのAzure AI Speech SDKリポジトリ](https://github.com/Azure-Samples/cognitive-services-speech-sdk/tree/master/sampledata/audiofiles)から取得できます。

In [7]:
# download sample audio file
import requests

sample_audio_url = "https://github.com/Azure-Samples/cognitive-services-speech-sdk/raw/master/sampledata/audiofiles/wikipediaOcelot.wav"
audio_file = requests.get(sample_audio_url)
with open("wikipediaOcelot.wav", "wb") as f:
    f.write(audio_file.content)

In [ ]:
transcription = client.audio.transcriptions.create(
    file=open("wikipediaOcelot.wav", "rb"),
    model=deployment,
)
print(transcription.text)